In [1]:
import pandas as pd
import numpy as np
import pyglicko2 as pg
win, loss, draw = pg.Outcome.WIN, pg.Outcome.LOSS, pg.Outcome.DRAW

# Calculate Glicko2 Ratings for F1 Drivers
This notebook calculates the Glicko2 (Elo-like) ratings for each F1 driver in each race using the `pyglicko2` library. For each race, only drivers who finished (have a numeric value in the `position` column) are considered. Each driver is compared against all other finishers in the same race, and ratings are updated accordingly.

In [22]:
race_df

,raceId,position,driverId
26971,1134,1,oscar-piastri
26972,1134,2,lando-norris
26973,1134,3,charles-leclerc
26974,1134,4,george-russell
26975,1134,5,nico-hulkenberg
26976,1134,6,lewis-hamilton
26977,1134,7,isack-hadjar
26978,1134,8,pierre-gasly
26979,1134,9,fernando-alonso
26980,1134,10,max-verstappen


In [109]:
class Player1(pg.Player):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.peak = self.r
        self.minimum = self.r

    def update_peak_min(self):
        self.peak = max(self.peak, self.r)
        self.minimum = min(self.minimum, self.r)

    def update(self, players, s):
        super().update(players, s)
        self.update_peak_min()

In [110]:
# Load race results
df = pd.read_csv('f1_res.csv')

# Only keep rows where 'position' is a number (driver finished the race)
df = df[pd.to_numeric(df['position'], errors='coerce').notnull()]
df['position'] = df['position'].astype(int)

# Get all unique driver IDs
driver_ids = df['driverId'].unique()

# Initialize Glicko2 players for each driver
players = {driver_id: Player1() for driver_id in driver_ids}

# Group by race
for race_id, race_df in df.groupby('raceId'):
    # Sort by finishing position
    race_df = race_df.sort_values('position')
    driver_list = race_df['driverId'].tolist()
    n = len(driver_list)
    print(race_id)
    # For each driver, only update against those they finished ahead of
    for i, driver in enumerate(driver_list):
        opponents = []
        scores = []
        for j, opp in enumerate(driver_list):
            if j <= i:
                continue  # Only consider drivers behind (i < j)
            if opp in players:
                opponents.append(players[opp])
                scores.append(win)
        for f in range(len(opponents)):
            try:
                pg.update(players[driver], opponents[f], win)
                # players[driver].update_peak()
                # print(driver, players[driver].r)

            except OverflowError:
                continue
            except ZeroDivisionError:
                continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [115]:
players_df = pd.DataFrame([
    {
        'driverId': driver_id,
        'glicko2_rating': player.r,
        # 'rd': player.rd,
        # 'vol': player.vol,
        'peak_rating': getattr(player, 'peak', player.peak),
        'minimum_rating': getattr(player, 'minimum', player.minimum)
    }
    for driver_id, player in players.items()
])
players_df

,driverId,glicko2_rating,peak_rating,minimum_rating
0,nino-farina,1840.439014,2100.940583,1500.000000
1,luigi-fagioli,1656.711963,1987.549741,1337.689107
2,reg-parnell,1606.771788,1701.661670,1243.299673
3,yves-giraud-cabantous,1114.087550,1550.859400,1098.362694
4,louis-rosier,1184.183591,1668.042516,1088.348889
...,...,...,...,...
673,franco-colapinto,1126.356046,1500.000000,1065.594182
674,jack-doohan,1044.281787,1500.000000,975.119755
675,andrea-kimi-antonelli,1358.880368,1780.139804,1358.880368
676,isack-hadjar,1532.853856,1532.853856,1055.134287


In [117]:
players_df.sort_values(by='peak_rating', ascending=False).to_csv('f1_glicko2.csv', index=False)